<a href="https://colab.research.google.com/github/manugaur99/ASD-classification-using-Large-margin-Softmax-Loss-for-stacked-2D-fMRI-data/blob/main/training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset
import torch
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
%matplotlib inline
np.random.seed(0)
torch.manual_seed(0)
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.metrics import classification_report, confusion_matrix
import pathlib
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import cv2
from google.colab.patches import cv2_imshow
from random import SystemRandom
import os
import torch
import torch.nn as nn
#from pytorchcv.model_provider import get_model as ptcv_get_model
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using =>", device)

In [ ]:
if torch.cuda.is_available():
  use_cuda = True
else:
  use_cuda = False
print(use_cuda)

In [ ]:
df_labels = pd.read_csv('/content/drive/MyDrive/ABIDE_NYU/phenotypes/Phenotypic_V1_0b_preprocessed1.csv')#path 

#df_labels = pd.read_csv('/content/drive/MyDrive/func_mean/Phenotypic_V1_0b_preprocessed1.csv')  ------ for jassi's drive 


#df_labels = df_labels[(df_labels['FILE_ID'].str[0:3].isin(["NYU"]))]
df_labels = df_labels[(df_labels['FILE_ID'].str[0:3].isin(["NYU"])) | (df_labels['FILE_ID'].str[0:3].isin(["USM"])) | (df_labels['FILE_ID'].str[0:6].isin(["UCLA_1"])) | (df_labels['FILE_ID'].str[0:4].isin(["UM_1"]))] # only NYU data
df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})      # 1: has autism  0: doesn't have autism
print(len(df_labels))

labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label

torch.save(labels, 'labels.pt')
print(labels)


In [ ]:
train_transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize((64,64)),
  transforms.RandomHorizontalFlip(p=0.30),
  transforms.RandomVerticalFlip(0.30),
  transforms.ToTensor(),
  transforms.Normalize([0.4103,0.4103,0.4103],[0.3906,0.3906,0.3906])
])
test_transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize((64,64)),
  transforms.ToTensor(),
  transforms.Normalize([0.4105,0.4105,0.4105], [0.3904,0.3904,0.3904])
  ])


In [ ]:
class AbideDataset(Dataset):
    def __init__(self, image_dir, labels, split):
        self.image_dir = image_dir       #Dataset/
        self.images = os.listdir(image_dir)
        self.labels = labels
        self.split = split 

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
    #   mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.load(img_path)           #73x61X3


        filename = self.images[index]
        #print(image.mean())
        #print("image is " , self.images[index] , "length is ", length)
        var = filename[:3]
        if var== "NYU" or var=="USM":
          key = filename[:11]
        elif var =="UCL":
          key = filename[:14]
        else:
          key = filename[:12]
 

        label = self.labels[key]
        if self.split== 'train':
          image = train_transform(image)
        else:
          image = test_transform(image)
          


        return image, label

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

train_dir = '/content/drive/MyDrive/func_mean/train/'
val_dir = '/content/drive/MyDrive/func_mean/val/'
test_dir = '/content/drive/MyDrive/func_mean/test/'


train_ds =  AbideDataset(image_dir= train_dir,labels =labels, split = 'train')
test_ds =  AbideDataset(image_dir= test_dir,labels =labels, split = 'test')
val_ds =  AbideDataset(image_dir= val_dir,labels =labels, split= 'val')


train_loader = DataLoader(  
        train_ds,
        batch_size=32,
        num_workers=2,
        pin_memory=True,
        shuffle=True,
    )
val_loader = DataLoader(
        val_ds,
        batch_size=32,
        num_workers=2,
        pin_memory=True,
        shuffle=False,
    )

test_loader = DataLoader(
        test_ds,
        batch_size=32,
        num_workers=2,
        pin_memory=True,
        shuffle=False,
    )

In [ ]:
train_count = train_ds.__len__()
val_count = val_ds.__len__()
test_count = test_ds.__len__()
print(train_count)
print(val_count)
print(test_count)


#CBAM module

In [ ]:
import torch.nn as nn   
import torch
import torch.nn.functional as F



class CBAM(nn.Module):

    def __init__(self, n_channels_in, reduction_ratio, kernel_size):
        super(CBAM, self).__init__()
        self.n_channels_in = n_channels_in
        self.reduction_ratio = reduction_ratio
        self.kernel_size = kernel_size

        self.channel_attention = ChannelAttention(n_channels_in, reduction_ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, f):
        #print("input to CBAM:     ", f.shape)
        chan_att = self.channel_attention(f)
        #print("Chann attn map:    ",chan_att.shape, f.shape)
        fp = chan_att * f
        #print("chann attn output: ",fp.shape)
        spat_att = self.spatial_attention(fp)
        #print("spatial attn map:  ",spat_att.shape)
        fpp = spat_att * fp
        #print("output by CBAM:    ", fpp.shape)
        return fpp


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size):
        super(SpatialAttention, self).__init__()
        self.kernel_size = kernel_size

        assert kernel_size % 2 == 1, "Odd kernel size required"
        self.conv = nn.Conv2d(in_channels = 2, out_channels = 1, kernel_size = kernel_size, padding= int((kernel_size-1)/2))
        # batchnorm

    def forward(self, x):
        max_pool = self.agg_channel(x, "max")
        avg_pool = self.agg_channel(x, "avg")
        pool = torch.cat([max_pool, avg_pool], dim = 1)
        conv = self.conv(pool)
        # batchnorm ????????????????????????????????????????????
        conv = conv.repeat(1,x.size()[1],1,1)
        att = torch.sigmoid(conv)        
        return att

    def agg_channel(self, x, pool = "max"):
        b,c,h,w = x.size()
        x = x.view(b, c, h*w)
        x = x.permute(0,2,1)
        if pool == "max":
            x = F.max_pool1d(x,c)
        elif pool == "avg":
            x = F.avg_pool1d(x,c)
        x = x.permute(0,2,1)
        x = x.view(b,1,h,w)
        return x


class ChannelAttention(nn.Module):
    def __init__(self, n_channels_in, reduction_ratio):
        super(ChannelAttention, self).__init__()
        self.n_channels_in = n_channels_in
        self.reduction_ratio = reduction_ratio
        self.middle_layer_size = int(self.n_channels_in/ float(self.reduction_ratio))

        self.bottleneck = nn.Sequential(
            nn.Linear(self.n_channels_in, self.middle_layer_size),
            nn.ReLU(),
            nn.Linear(self.middle_layer_size, self.n_channels_in)
        )


    def forward(self, x):
        kernel = (x.size()[2], x.size()[3])
        avg_pool = F.avg_pool2d(x, kernel )
        max_pool = F.max_pool2d(x, kernel)

        
        avg_pool = avg_pool.view(avg_pool.size()[0], -1)
        max_pool = max_pool.view(max_pool.size()[0], -1)
        

        avg_pool_bck = self.bottleneck(avg_pool)
        max_pool_bck = self.bottleneck(max_pool)

        pool_sum = avg_pool_bck + max_pool_bck

        sig_pool = torch.sigmoid(pool_sum)
        sig_pool = sig_pool.unsqueeze(2).unsqueeze(3)

        out = sig_pool.repeat(1,1,kernel[0], kernel[1])
        return out

def main():
    # ca = CBAM() 


    f = torch.FloatTensor([
        [
            [[1,1,1,1,1], [1,1,2,1,1], [1,1,1,1,1]],
            [[2,2,2,2,2], [2,2,3,2,2], [2,2,2,2,2]],
            [[3,3,3,3,3], [3,3,4,3,3], [3,3,3,3,3]]
        ]
    ])

    #print(f.size())

    # sa = SpatialAttention(kernel_size = 3)
    # sa(f)
    cbam = CBAM(n_channels_in = f.size()[1],reduction_ratio = 2, kernel_size = 3) 


    fpp = cbam(f)
    print(fpp.size())
    print(fpp)
    # print(f)
    # print(fp)
    


if __name__ == "__main__":
    main()

In [ ]:
x = torch.FloatTensor(2048,2)
x.shape
y = nn.Parameter(x)
y.shape

#L-Softmax

In [ ]:
import math
import torch
from torch import nn
from scipy.special import binom


class LSoftmaxLinear(nn.Module):

    def __init__(self, input_features, output_features, margin, device):
        super().__init__()
        self.input_dim = input_features  # number of input feature i.e. output of the last fc layer
        self.output_dim = output_features  # number of output = class numbers
        self.margin = margin  # m
        self.beta = 100
        self.beta_min = 0
        self.scale = 0.99

        self.device = device  # gpu or cpu

        # Initialize L-Softmax parameters
        self.weight = nn.Parameter(torch.FloatTensor(input_features, output_features))
        self.divisor = math.pi / self.margin  # pi/m
        self.C_m_2n = torch.Tensor(binom(margin, range(0, margin + 1, 2))).to(device)  # C_m{2n}
        self.cos_powers = torch.Tensor(range(self.margin, -1, -2)).to(device)  # m - 2n
        self.sin2_powers = torch.Tensor(range(len(self.cos_powers))).to(device)  # n
        self.signs = torch.ones(margin // 2 + 1).to(device)  # 1, -1, 1, -1, ...
        self.signs[1::2] = -1

    def calculate_cos_m_theta(self, cos_theta):
        sin2_theta = 1 - cos_theta**2
        cos_terms = cos_theta.unsqueeze(1) ** self.cos_powers.unsqueeze(0)  # cos^{m - 2n}
        sin2_terms = (sin2_theta.unsqueeze(1)  # sin2^{n}
                      ** self.sin2_powers.unsqueeze(0))

        cos_m_theta = (self.signs.unsqueeze(0) *  # -1^{n} * C_m{2n} * cos^{m - 2n} * sin2^{n}
                       self.C_m_2n.unsqueeze(0) *
                       cos_terms *
                       sin2_terms).sum(1)  # summation of all terms

        return cos_m_theta

    def reset_parameters(self):
        nn.init.kaiming_normal_(self.weight.data.t())

    def find_k(self, cos):
        # to account for acos numerical errors
        eps = 1e-7
        cos = torch.clamp(cos, -1 + eps, 1 - eps)
        acos = cos.acos()
        k = (acos / self.divisor).floor().detach()
        return k

    def forward(self, input, target=None):
        if self.training:
            assert target is not None
            x, w = input, self.weight
            #print("input shape :",x.shape)
            #print("weight shape:", w.shape)
            beta = max(self.beta, self.beta_min)
            logit = x.mm(w)
            #print("matrix multiplication to get logit : ", logit.shape)
            indexes = range(logit.size(0))
            #print("indexes: ", indexes.shape)
            logit_target = logit[indexes, target]
            #print("logit_taget shape: ",logit_target.shape)

            # cos(theta) = w * x / ||w||*||x||
            w_target_norm = w[:, target].norm(p=2, dim=0)
            x_norm = x.norm(p=2, dim=1)
            cos_theta_target = logit_target / (w_target_norm * x_norm + 1e-10)

            # equation 7
            cos_m_theta_target = self.calculate_cos_m_theta(cos_theta_target)

            # find k in equation 6
            k = self.find_k(cos_theta_target)

            # f_y_i
            logit_target_updated = (w_target_norm *
                                    x_norm *
                                    (((-1) ** k * cos_m_theta_target) - 2 * k))
            logit_target_updated_beta = (logit_target_updated + beta * logit[indexes, target]) / (1 + beta)

            logit[indexes, target] = logit_target_updated_beta
            self.beta *= self.scale
            return logit
        else:
            #assert target is None
            return input.mm(self.weight)

#Modifying model for 64,64 images

In [ ]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''

# taken from https://github.com/kuangliu/pytorch-cifar
import torch
import torch.nn as nn
import torch.nn.functional as F



# class BasicBlock(nn.Module):
#     expansion = 1

#     def __init__(self, in_planes, planes, stride=1, reduction_ratio = 1, kernel_cbam = 3, use_cbam = False):
#         super(BasicBlock, self).__init__()
#         self.use_cbam = use_cbam
#         self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(planes)

#         if self.use_cbam:
#             self.cbam = CBAM(n_channels_in = self.expansion*planes, reduction_ratio = reduction_ratio, kernel_size = kernel_cbam)


#         self.shortcut = nn.Sequential()
#         if stride != 1 or in_planes != self.expansion*planes:
#             self.shortcut = nn.Sequential(
#                 nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(self.expansion*planes)
#             )


#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))

#         #cbam
#         if self.use_cbam:
#             out = self.cbam(out)

#         out += self.shortcut(x)
#         out = F.relu(out)
#         return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, reduction_ratio = 16, kernel_cbam = 3, use_cbam = False):
        super(Bottleneck, self).__init__()
        self.use_cbam = use_cbam

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

        if self.use_cbam:
            self.cbam = CBAM(n_channels_in = self.expansion*planes, reduction_ratio = reduction_ratio, kernel_size = kernel_cbam)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        #cbam
        if self.use_cbam:
            out = self.cbam(out)

        out += self.shortcut(x)
        out = F.relu(out)
        
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2, reduction_ratio = 16, kernel_cbam = 3, use_cbam_block= False, use_cbam_class = False):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.reduction_ratio = reduction_ratio
        self.kernel_cbam = kernel_cbam
        self.use_cbam_block = use_cbam_block
        self.use_cbam_class = use_cbam_class

        #print(use_cbam_block, use_cbam_class)

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
#--------------------------------------------------------------------------------------       
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
       # self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.lsoftmax_linear = LSoftmaxLinear(input_features=2048, output_features=2, margin=3, device= device)
       # self.softmax = nn.Softmax(dim = 1)
#------------------------------------------------------------------------------------------
        if self.use_cbam_class:
            self.cbam = CBAM(n_channels_in = 512*block.expansion, reduction_ratio = reduction_ratio, kernel_size = kernel_cbam)


    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, self.reduction_ratio, self.kernel_cbam, self.use_cbam_block))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, labels):
        #print("input shape:             ",x.shape)
        out = F.relu(self.bn1(self.conv1(x)))
        #print("Conv1 output:            ",out.shape)
        out = self.maxpool(out)
        #print("maxpool output:          ",out.shape)

        out = self.layer1(out)
        #print("----------LAYER 1 output:",out.shape)
        out = self.layer2(out)
        #print("----------LAYER 2 output:",out.shape)
        out = self.layer3(out)
        #print("----------LAYER 3 output:",out.shape)
        out = self.layer4(out)
        #print("----------LAYER 4 output:",out.shape)

        if self.use_cbam_class:
            out = out  + self.cbam(out)
            #print("CBAM output:         ",out.shape)
        out = self.avgpool(out)
        #print("AVGpool output:          ",out.shape)
        out = torch.flatten(out, 1)
        #print("flattened shape:         ",out.shape)
        #out = self.fc(out)
        #print("final linear layer:      ",out)
        out = self.lsoftmax_linear(input=out, target=labels)
        #out = self.softmax(out)
        #print("after softmax:           ",out)


#----------------------------------------------------------------------------------------------------------------------
        #out = F.avg_pool2d(out, 4)
        #out = out.view(out.size(0), -1)
        #print("view changed;            ",out.shape)
      
        #out = self.linear(out)
        #print("final linear layer:      ",out.shape)
        return out




# def ResNet18(reduction_ratio = 1, kernel_cbam = 3, use_cbam_block = False, use_cbam_class = False):
#     print(kernel_cbam)
#     return ResNet(
#                 BasicBlock, 
#                 [2,2,2,2], 
#                 reduction_ratio= reduction_ratio,
#                 kernel_cbam = kernel_cbam,
#                 use_cbam_block= use_cbam_block,
#                 use_cbam_class = use_cbam_class
#                 )

# def ResNet34(reduction_ratio = 1, kernel_cbam = 3, use_cbam_block = False, use_cbam_class = False):
#     return ResNet(
#         BasicBlock,
#         [3,4,6,3],
#         reduction_ratio= reduction_ratio, 
#         kernel_cbam = kernel_cbam, 
#         use_cbam_block= use_cbam_block,
#         use_cbam_class = use_cbam_class
#         )

def ResNet50(reduction_ratio = 16, kernel_cbam = 3, use_cbam_block = True, use_cbam_class = True):
    return ResNet(
        Bottleneck, 
        [3,4,6,3], 
        reduction_ratio= reduction_ratio, 
        kernel_cbam = kernel_cbam, 
        use_cbam_block= use_cbam_block,
        use_cbam_class = use_cbam_class
        )

def ResNet101(reduction_ratio = 16, kernel_cbam = 3, use_cbam_block = True, use_cbam_class = True):
    return ResNet(
        Bottleneck, 
        [3,4,23,3], 
        reduction_ratio= reduction_ratio, 
        kernel_cbam = kernel_cbam, 
        use_cbam_block= use_cbam_block,
        use_cbam_class = use_cbam_class
        )

def ResNet152(reduction_ratio = 1, kernel_cbam = 3, use_cbam_block = False, use_cbam_class = False):
    return ResNet(
        Bottleneck, 
        [3,8,36,3], 
        reduction_ratio= reduction_ratio, 
        kernel_cbam = kernel_cbam, 
        use_cbam_block= use_cbam_block,
        use_cbam_class = use_cbam_class
        )

def ResNetk(k, reduction_ratio = 1, kernel_cbam = 3, use_cbam_block = True, use_cbam_class = True ):
    possible_depth = [18,34,50,101,152]
    assert k in possible_depth, "Choose a depth in {}".format(possible_depth)

#    if k == 18:
#        return ResNet18(reduction_ratio= reduction_ratio, kernel_cbam = kernel_cbam, use_cbam_block= use_cbam_block, use_cbam_class = use_cbam_class)
#    elif k == 34:
#        return ResNet34(reduction_ratio= reduction_ratio, kernel_cbam = kernel_cbam, use_cbam_block= use_cbam_block, use_cbam_class = use_cbam_class)
    if k == 50:
        return ResNet50(reduction_ratio= reduction_ratio, kernel_cbam = kernel_cbam, use_cbam_block= use_cbam_block, use_cbam_class = use_cbam_class)
    elif k == 101:
        return ResNet101(reduction_ratio= reduction_ratio, kernel_cbam = kernel_cbam, use_cbam_block= use_cbam_block, use_cbam_class = use_cbam_class)
    elif k == 152:
        return ResNet152(reduction_ratio= reduction_ratio, kernel_cbam = kernel_cbam, use_cbam_block= use_cbam_block, use_cbam_class = use_cbam_class)




In [ ]:
# !pip install modelsummary

In [ ]:
model = ResNet50()
tensor = torch.rand(1,3,64,64)
#model(tensor)
model = model.to(device=device)

In [ ]:

# from modelsummary import summary

# net = ResNet50()


# summary(net, torch.zeros((1, 3, 64, 64)), show_input=True)

# Hinton's margin loss

In [ ]:
def margin_loss(x, labels):
    #m = nn.Softmax(dim=1)
    batch_size = x.size(0)
    #print("batch size", batch_size) 
    #x = m(x)
    v_c = torch.sqrt((x ** 2).sum(dim=1, keepdim=True))
    #print("v_c shape", v_c.shape)

    left = F.relu(0.9 - v_c).view(batch_size, -1)
    right = F.relu(v_c - 0.1).view(batch_size, -1)
    #print("left shape", left.shape)
    #print("right shape", right.shape)
    loss = labels * left + 0.5 * (1.0 - labels) * right
    loss = loss.sum(dim=1).mean()
    
    #print("margin loss",loss)
    #print("loss shape = ",loss.shape)


    return  loss


In [ ]:
loss_function = nn.CrossEntropyLoss()
#loss_function = torch.nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr= 0.01, momentum=0.9)
#optimizer=Adam(model.parameters(),lr=0.01,weight_decay=0.001)

#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=25, verbose = True)
num_epochs = 150
alpha = 0.4

#Mix up Training

model(images) >> model(images, labels) for L-softmax

In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
best_accuracy=0.0
num_epochs = 300
for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    correct = 0
    total =  0 
    metric = 0 
    
    for i, (images,labels) in enumerate(train_loader):
      if torch.cuda.is_available():
        images= images.cuda().float()
        labels=labels.cuda()
      images, labels_a, labels_b, lam = mixup_data(images, labels,
                                                       alpha, use_cuda)
      images, labels_a, labelss_b = map(Variable, (images,
                                                      labels_a, labels_b))            
      optimizer.zero_grad()

      
      outputs=model(images,labels)
      loss = mixup_criterion(loss_function, outputs, labels_a, labels_b, lam)
      train_loss+= loss.cpu().data*images.size(0)
      _,prediction=torch.max(outputs.data,1)

      total += labels.size(0)

      correct += (lam * prediction.eq(labels_a.data).cpu().sum().float()
                    + (1 - lam) * prediction.eq(labels_b.data).cpu().sum().float())
      loss.backward()
      optimizer.step()
        
        
      
        
      #train_accuracy+=int(torch.sum(prediction==labels.data))
# mix up train accuracy
    train_accuracy = 100.*correct/total       
    #train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
     # Evaluation on validation dataset
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
      val_accuracy = 0.0
      for i, (images,labels) in enumerate(val_loader):
        if torch.cuda.is_available():
          images=Variable(images.cuda()).float()
          labels=Variable(labels.cuda())
           #L-softmax   
        outputs=model(images,None)
        _,prediction=torch.max(outputs.data,1)
        val_accuracy+=int(torch.sum(prediction==labels.data))
      
      val_accuracy=val_accuracy/val_count
      
      #scheduler.step(metric)

      print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' val Accuracy: '+str(val_accuracy))
      #print('lr:', scheduler.get_last_lr())
      
      # #Save the best model
      
      if val_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=val_accuracy

In [ ]:
print(best_accuracy) 

In [ ]:
import copy
model.load_state_dict(torch.load('best_checkpoint.model'))
model2 =copy.deepcopy(model)

In [ ]:
model.load_state_dict(torch.load('best_checkpoint.model'))

In [ ]:
for i in range(20):
  model.eval()
  test_loss = 0.0
  with torch.no_grad():
    test_accuracy = 0.0
    for i, (images,labels) in enumerate(test_loader):
      if torch.cuda.is_available():
        images=Variable(images.cuda()).float()
        labels=Variable(labels.cuda())
            
      outputs=model(images,None)
      _,prediction=torch.max(outputs.data,1)
      test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    #scheduler.step(metric)

    print(' Test Loss: '+str(test_loss)+' Test Accuracy: '+str(test_accuracy))
    #print('lr:', scheduler.get_last_lr())
    